# Object detection using region proposal (selective search)

Region proposal algorithms tend to be far more efficient than the traditional object detection techniques of image pyramids and sliding windows because:

- Fewer individual ROIs are examined

- It is faster than exhaustively examining every scale/location of the input image

- The amount of accuracy lost is minimal, if any

In [1]:
#--Tensorflow
from tensorflow.keras.applications import ResNet50, imagenet_utils
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

#--Others
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import cv2

### Define the selective search function

In [2]:
def selective_search(image, method="fast"):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)

    #Check to see which method we are using. Either "fast" or "quality"
    if method == "fast":
        ss.switchToSelectiveSearchFast()
    
    else:
        ss.switchToSelectiveSearchQuality()
    
    #Run selective search on the image
    rects = ss.process()

    return rects

In [3]:
#Initialize the filters 
labelFilters = None

#If the varaible is not empty, break it into a list
if labelFilters is not None:
    labelFilters = labelFilters.lower().split("")

### Load the model

In [4]:
model = ResNet50(weights="imagenet")

In [5]:
#Load the input image
image = cv2.imread('/media/juan/juan1/pyimage_univ/object_detect_201/region-proposal-object-detection/beagle.png')
(H, W) = image.shape[:2]

In [6]:
#Run the selective search
recs = selective_search(image, "fast")

In [7]:
#Initialize two lists in order to store the coordinates and ROIs
rois, boxes = [], []

In [8]:
#loop through the results from the selective search
for (x, y, w, h) in recs:
    #Filter small objects that can be noise.
    #For this, check if the width and height of the region is less than 10% of 
    #the image width or height, ignore it

    if w / float(W) < 0.1 or h / float(H)<0.1:
        continue

    #Extract the ROI from the input image. Change to RGB and resize
    roi = image[y:y+h, x:x+w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224,224))

    #Further process the roi image so it can be fed to the resnet50 model
    roi = img_to_array(roi)
    roi = preprocess_input(roi)

    #Update proposals and bounding boxes list
    rois.append(roi)
    boxes.append((x, y, w, h))

In [9]:
#Convert proposals to numpy arrays in order to make them compatible to the model
proposals = np.array(rois, dtype="float32")

### Make predictions

In [10]:
preds = model.predict(proposals)
preds = imagenet_utils.decode_predictions(preds, top=1)

17/17 [==============================] - 5s 168ms/step


### Get the predictions and assign them a label

In [11]:
#Initialize an empty label dictionary
labels = {}

In [12]:
for (i,p) in enumerate(preds):
    (ImagenetID, label, prob) = p[0]

    #Only if the label filters are not empty and the label does not exist
    if labelFilters is not None and label not in labelFilters:
        continue

    if prob >= 0.95:
        #grab the bounding box associated with the prediction and covert coordinates
        (x, y, w, h) = boxes[i]
        box = (x, y, x+w, y+h)

        #grab the lust of predictions for the label and add te bounding box
        L = labels.get(label, [])
        L.append((box, prob))
        labels[label] = L

In [16]:
#loop thorugh the labels for each of the detected objects in the image
for label in labels.keys():
    #clone the image so to draw on it
    clone = image.copy()

    for (box, prob) in labels[label]:
        #draw bounding box
        (x, y, w, h) =box
        cv2.rectangle(clone, (x,y), (w, h), (0,255,0), 2)

    #Show the image
    cv2.imshow('result before nms', clone)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    clone2 = image.copy()

    #Extract the bounding boxes and apply nms
    boxes = np.array([p[0] for p in labels[label]])
    proba = np.array([p[1] for p in labels[label]])
    boxes = non_max_suppression(boxes, proba)

    #loop over all bounding boxes that were kept
    for (startX, startY, endX, endY) in boxes:
        #Draw them
        cv2.rectangle(clone2, (startX, startY), (endX, endY), (0,255,0), 2)
        #Arrange the position of the text so is not overlapped with the box
        y = startY -10 if startY-10 > 10 else startY+10
        cv2.putText(clone2, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45,
        (0,255,0), 2)

    cv2.imshow("after nms", clone2)
    cv2.waitKey(0)
cv2.destroyAllWindows() 